In [1]:
import cudf
import cuml

import cupy as cp

In [2]:
infected_df = cudf.read_csv('./data/week1.csv', usecols=['lat', 'long', 'infected'])


In [3]:
infected_df = infected_df[infected_df['infected']==True].reset_index(drop=True)
infected_df


,lat,long,infected
0,54.472766,-1.654932,True
1,54.529717,-1.667143,True
2,54.512986,-1.589866,True
3,54.522322,-1.380694,True
4,54.541660,-1.613490,True
...,...,...,...
18143,52.428347,-3.322932,True
18144,52.415895,-3.263942,True
18145,52.539934,-3.617128,True
18146,52.435490,-3.597263,True


In [4]:
# https://www.ordnancesurvey.co.uk/docs/support/guide-coordinate-systems-great-britain.pdf

def latlong2osgbgrid_cupy(lat, long, input_degrees=True):
    '''
    Converts latitude and longitude (ellipsoidal) coordinates into northing and easting (grid) coordinates, using a Transverse Mercator projection.
    
    Inputs:
    lat: latitude coordinate (N)
    long: longitude coordinate (E)
    input_degrees: if True (default), interprets the coordinates as degrees; otherwise, interprets coordinates as radians
    
    Output:
    (northing, easting)
    '''
    
    if input_degrees:
        lat = lat * cp.pi/180
        long = long * cp.pi/180

    a = 6377563.396
    b = 6356256.909
    e2 = (a**2 - b**2) / a**2

    N0 = -100000 # northing of true origin
    E0 = 400000 # easting of true origin
    F0 = .9996012717 # scale factor on central meridian
    phi0 = 49 * cp.pi / 180 # latitude of true origin
    lambda0 = -2 * cp.pi / 180 # longitude of true origin and central meridian
    
    sinlat = cp.sin(lat)
    coslat = cp.cos(lat)
    tanlat = cp.tan(lat)
    
    latdiff = lat-phi0
    longdiff = long-lambda0

    n = (a-b) / (a+b)
    nu = a * F0 * (1 - e2 * sinlat ** 2) ** -.5
    rho = a * F0 * (1 - e2) * (1 - e2 * sinlat ** 2) ** -1.5
    eta2 = nu / rho - 1
    M = b * F0 * ((1 + n + 5/4 * (n**2 + n**3)) * latdiff - 
                  (3*(n+n**2) + 21/8 * n**3) * cp.sin(latdiff) * cp.cos(lat+phi0) +
                  15/8 * (n**2 + n**3) * cp.sin(2*(latdiff)) * cp.cos(2*(lat+phi0)) - 
                  35/24 * n**3 * cp.sin(3*(latdiff)) * cp.cos(3*(lat+phi0)))
    I = M + N0
    II = nu/2 * sinlat * coslat
    III = nu/24 * sinlat * coslat ** 3 * (5 - tanlat ** 2 + 9 * eta2)
    IIIA = nu/720 * sinlat * coslat ** 5 * (61-58 * tanlat**2 + tanlat**4)
    IV = nu * coslat
    V = nu / 6 * coslat**3 * (nu/rho - cp.tan(lat)**2)
    VI = nu / 120 * coslat ** 5 * (5 - 18 * tanlat**2 + tanlat**4 + 14 * eta2 - 58 * tanlat**2 * eta2)

    northing = I + II * longdiff**2 + III * longdiff**4 + IIIA * longdiff**6
    easting = E0 + IV * longdiff + V * longdiff**3 + VI * longdiff**5

    return(northing, easting)

In [5]:
cupy_lat = cp.asarray(infected_df['lat'])
cupy_long = cp.asarray(infected_df['long'])

In [6]:
n_cupy_array, e_cupy_array = latlong2osgbgrid_cupy(cupy_lat, cupy_long)
infected_df['northing'] = cudf.Series(n_cupy_array).astype('float32')
infected_df['easting'] = cudf.Series(e_cupy_array).astype('float32')
print(infected_df.dtypes)
infected_df.head()

lat         float64
long        float64
infected       bool
northing    float32
easting     float32
dtype: object


,lat,long,infected,northing,easting
0,54.472766,-1.654932,True,508670.06250,422359.7500
1,54.529717,-1.667143,True,515002.65625,421538.5625
2,54.512986,-1.589866,True,513167.53125,426549.8750
3,54.522322,-1.380694,True,514305.28125,440081.2500
4,54.541660,-1.613490,True,516349.12500,425003.0000


In [7]:
infected_df['easting'].mean()

387128.2195748154

In [8]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}